# Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import plotly.express as px
import plotly.graph_objects as go
import os
from tqdm import tqdm
from torch.utils.data import DataLoader

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [3]:
def train_test_divide (data_x, data_x_hat, data_t, data_t_hat, train_rate = 0.8):
  """Divide train and test data for both original and synthetic data.

  Args:
    - data_x: original data
    - data_x_hat: generated data
    - data_t: original time
    - data_t_hat: generated time
    - train_rate: ratio of training data from the original data
  """
  # Divide train/test index (original data)
  no = len(data_x)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]

  train_x = [data_x[i] for i in train_idx]
  test_x = [data_x[i] for i in test_idx]
  train_t = [data_t[i] for i in train_idx]
  test_t = [data_t[i] for i in test_idx]

  # Divide train/test index (synthetic data)
  no = len(data_x_hat)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]

  train_x_hat = [data_x_hat[i] for i in train_idx]
  test_x_hat = [data_x_hat[i] for i in test_idx]
  train_t_hat = [data_t_hat[i] for i in train_idx]
  test_t_hat = [data_t_hat[i] for i in test_idx]

  return train_x, train_x_hat, test_x, test_x_hat, train_t, train_t_hat, test_t, test_t_hat


def extract_time (data):
  """Returns Maximum sequence length and each sequence length.

  Args:
    - data: original data

  Returns:
    - time: extracted time information
    - max_seq_len: maximum sequence length
  """
  time = list()
  max_seq_len = 0
  for i in range(len(data)):
    max_seq_len = max(max_seq_len, len(data[i][:,0]))
    time.append(len(data[i][:,0]))

  return time, max_seq_len

def random_generator (batch_size, z_dim, T_mb, max_seq_len):
  """Random vector generation.

  Args:
    - batch_size: size of the random vector
    - z_dim: dimension of random vector
    - T_mb: time information for the random vector
    - max_seq_len: maximum sequence length

  Returns:
    - Z_mb: generated random vector
  """
  Z_mb = list()
  for i in range(batch_size):
    temp = np.zeros([max_seq_len, z_dim])
    temp_Z = np.random.uniform(0., 1, [T_mb[i], z_dim])
    temp[:T_mb[i],:] = temp_Z
    Z_mb.append(temp_Z)
  return Z_mb


def batch_generator(data, time, batch_size):
  """Mini-batch generator.

  Args:
    - data: time-series data
    - time: time information
    - batch_size: the number of samples in each batch

  Returns:
    - X_mb: time-series data in each batch
    - T_mb: time information in each batch
  """
  no = len(data)
  idx = np.random.permutation(no)
  train_idx = idx[:batch_size]

  X_mb = list(data[i] for i in train_idx)
  T_mb = list(time[i] for i in train_idx)

  return X_mb, T_mb

In [4]:
class Time_GAN_module(nn.Module):
    """
    Class from which a module of the Time GAN Architecture can be constructed,
    consisting of a n_layer stacked RNN layers and a fully connected layer

    input_size = dim of data (depending if module operates on latent or non-latent space)
    """
    def __init__(self, input_size, output_size, hidden_dim, n_layers, activation=torch.sigmoid):
        super(Time_GAN_module, self).__init__()

        # Parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sigma = activation

        #Defining the layers
        # RNN Layer
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True, device=device)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size, device=device)

    def forward(self, x):

            batch_size = x.size(0)

            # Initializing hidden state for first input using method defined below
            hidden = self.init_hidden(batch_size)

            # Passing in the input and hidden state into the model and obtaining outputs
            out, hidden = self.rnn(x, hidden)

            # Reshaping the outputs such that it can be fit into the fully connected layer
            out = out.contiguous().view(-1, self.hidden_dim)
            out = self.fc(out)

            if self.sigma == nn.Identity:
                idendity = nn.Identity()
                return idendity(out)

            out = self.sigma(out)

            # HIDDEN STATES WERDEN IN DER PAPER IMPLEMENTIERUNG AUCH COMPUTED, ALLERDINGS NICHT BENUTZT?

            return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim, device=device)
        return hidden

In [5]:
def Normalize(dta):
  return (dta - np.min(dta, 0)) /  (np.max(dta, 0) - np.min(dta, 0) + 1e-7)

In [6]:
def real_data_loading (path, seq_len):
  """Load and preprocess real-world datasets.

  Args:
    - data_name: stock or energy
    - seq_len: sequence length

  Returns:
    - data: preprocessed data.
  """
  assert os.path.isfile(path)
  ori_data = np.loadtxt(path, delimiter = ",",skiprows = 1)

  # Flip the data to make chronological data
  ori_data = ori_data[::-1]
  # Normalize the data
  ori_data = Normalize(ori_data)

  # Preprocess the dataset
  temp_data = []
  # Cut data by sequence length
  for i in range(0, len(ori_data) - seq_len):
    _x = ori_data[i:i + seq_len]
    # yields an array of dims [len(dta) - seq_len, seq_len, n_variables]
    temp_data.append(_x)

  # Mix the datasets (to make it similar to i.i.d)
  idx = np.random.permutation(len(temp_data))
  data = []
  for i in range(len(temp_data)):
    data.append(temp_data[idx[i]])

  return data

In [7]:
file = '/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/stock_data.csv'
seq_len = 24
data = real_data_loading(file, seq_len)
data = torch.tensor(np.array(data), device=device, dtype=torch.float32).detach()

In [8]:
file = '/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/GOOG.csv'
seq_len = 24
goog = real_data_loading(file, seq_len)
goog = torch.tensor(np.array(goog), device=device, dtype=torch.float32).detach()

In [9]:
input_size = 6
output_size = 24
hidden_dim = 24
n_layers = 3
gamma = 1

no, seq_len, dim = len(data), 24, 6

batch_size = 128
epoch = 50

In [10]:
Embedder = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers).to(device)
Recovery = Time_GAN_module(input_size=hidden_dim, output_size=dim, hidden_dim=hidden_dim, n_layers=n_layers).to(device)
Supervisor = Time_GAN_module(input_size=hidden_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers-1).to(device)
Embedder.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Embedder.pth'))
Supervisor.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Supervisor.pth'))
Recovery.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Recovery.pth'))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
next(Embedder.parameters()).device

# Treinamento conjunto

In [58]:
random_data = random_generator(batch_size=batch_size, z_dim=dim,
                                       T_mb=extract_time(data)[0], max_seq_len=extract_time(data)[1])
random_data = torch.tensor(np.array(random_data))

In [87]:
loader = DataLoader(data, batch_size, shuffle=True, drop_last=True)
random_loader = DataLoader(random_data, batch_size, shuffle=True, drop_last=True)
binary_cross_entropy_loss = nn.BCEWithLogitsLoss()
MSE_loss = nn.MSELoss()

In [13]:
parameters = dict()
parameters['module'] = 'gru'
parameters['hidden_dim'] = 24
parameters['num_layers'] = 3
parameters['iterations'] = 10000
parameters['batch_size'] = 128
parameters['epoch'] = 1000

In [14]:
hidden_dim = parameters["hidden_dim"]
num_layers = parameters["num_layers"]
iterations = parameters["iterations"]
batch_size = parameters["batch_size"]
module = parameters["module"]
epoch = parameters["epoch"]
no, seq_len, dim = data.shape
z_dim = dim
gamma = 1

In [78]:
Embedder = Time_GAN_module(input_size=z_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=num_layers)
Recovery = Time_GAN_module(input_size=hidden_dim, output_size=dim, hidden_dim=hidden_dim, n_layers=n_layers)
Generator = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
Supervisor = Time_GAN_module(input_size=hidden_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers-1)
Discriminator = Time_GAN_module(input_size=hidden_dim, output_size=1,
                                hidden_dim=hidden_dim, n_layers=n_layers, activation=nn.Identity)

embedder_optimizer = optim.Adam(Embedder.parameters(), lr=0.001)
recovery_optimizer = optim.Adam(Recovery.parameters(), lr=0.001)
supervisor_optimizer = optim.Adam(Recovery.parameters(), lr=0.001)
discriminator_optimizer = optim.Adam(Discriminator.parameters(), lr=0.001)
generator_optimizer = optim.Adam(Generator.parameters(), lr=0.001)

In [90]:
print('Start Embedding Network Training')
valLoss = []
trainLoss = []
for e in range(epoch):
    for batch_index, X in enumerate(loader):
        Embedder.train()
        Recovery.train()

        MSE_loss = nn.MSELoss()

        H, _ = Embedder(X.float())
        H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

        X_tilde, _ = Recovery(H)
        X_tilde = torch.reshape(X_tilde, (batch_size, seq_len, dim))

        E_loss0 = 10 * torch.sqrt(MSE_loss(X, X_tilde))

        Embedder.zero_grad()
        Recovery.zero_grad()

        E_loss0.backward(retain_graph=True)

        embedder_optimizer.step()
        recovery_optimizer.step()

        if e in range(1,epoch) and batch_index == 0:
            Embedder.eval()
            Recovery.eval()
            H, _ = Embedder(goog)
            H = torch.reshape(H, (goog.shape[0], seq_len, hidden_dim))
            X_tilde, _ = Recovery(H)
            X_tilde = torch.reshape(X_tilde, (goog.shape[0], seq_len, dim))
            valLoss.append(10 * torch.sqrt(MSE_loss(goog, X_tilde)).cpu().detach().numpy())
            H, _ = Embedder(data)
            H = torch.reshape(H, (data.shape[0], seq_len, hidden_dim))
            X_tilde, _ = Recovery(H)
            X_tilde = torch.reshape(X_tilde, (data.shape[0], seq_len, dim))
            trainLoss.append(10 * torch.sqrt(MSE_loss(data, X_tilde)).cpu().detach().numpy())
            print('step: '+ str(e) + '/' + str(epoch) + ', e_loss: ' + str(np.sqrt(E_loss0.cpu().detach().numpy())))

print('Finish Embedding Network Training')

Start Embedding Network Training
step: 1/1000, e_loss: 1.4258364
step: 2/1000, e_loss: 1.1318593
step: 3/1000, e_loss: 0.8649914
step: 4/1000, e_loss: 0.72856146
step: 5/1000, e_loss: 0.60473526
step: 6/1000, e_loss: 0.61664253
step: 7/1000, e_loss: 0.59440136
step: 8/1000, e_loss: 0.5839859
step: 9/1000, e_loss: 0.53621507
step: 10/1000, e_loss: 0.56508905
step: 11/1000, e_loss: 0.56084764
step: 12/1000, e_loss: 0.5033022
step: 13/1000, e_loss: 0.5452877
step: 14/1000, e_loss: 0.5184208
step: 15/1000, e_loss: 0.5597326
step: 16/1000, e_loss: 0.5518124
step: 17/1000, e_loss: 0.51220596
step: 18/1000, e_loss: 0.49391752
step: 19/1000, e_loss: 0.5215997
step: 20/1000, e_loss: 0.5002607
step: 21/1000, e_loss: 0.53171027
step: 22/1000, e_loss: 0.5101206
step: 23/1000, e_loss: 0.47555798
step: 24/1000, e_loss: 0.49282762
step: 25/1000, e_loss: 0.49965364
step: 26/1000, e_loss: 0.4835782
step: 27/1000, e_loss: 0.47366017
step: 28/1000, e_loss: 0.48332524
step: 29/1000, e_loss: 0.5099566
step

In [91]:
fig = px.line(trainLoss)
fig.add_trace(go.Scatter(y=valLoss, mode="lines"))
fig.show()

In [92]:
H, _ = Embedder(goog)
H = torch.reshape(H, (goog.shape[0], seq_len, hidden_dim))
X_tilde, _ = Recovery(H)
X_tilde = torch.reshape(X_tilde, (goog.shape[0], seq_len, dim))

In [93]:
px.line(X_tilde[0].cpu().detach())

In [95]:
px.line(goog[0].cpu().detach())

In [97]:
print('Start Training with Supervised Loss Only')
valLoss = []
trainLoss = []
for e in range(epoch):
    for batch_index, X in enumerate(loader):
        Embedder.train()
        Supervisor.train()

        H, _ = Embedder(X.float())
        H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

        H_hat_supervise, _ = Supervisor(H)
        H_hat_supervise = torch.reshape(H_hat_supervise, (batch_size, seq_len, hidden_dim))

        G_loss_S = MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:])

        Embedder.zero_grad()
        Supervisor.zero_grad()

        G_loss_S.backward(retain_graph=True)

        embedder_optimizer.step()
        supervisor_optimizer.step()

        if e in range(1,epoch) and batch_index == 0:
            Embedder.eval()
            Supervisor.eval()
            H, _ = Embedder(goog)
            H = torch.reshape(H, (goog.shape[0], seq_len, hidden_dim))
            H_hat_supervise, _ = Supervisor(H)
            H_hat_supervise = torch.reshape(H_hat_supervise, (goog.shape[0], seq_len, hidden_dim))
            valLoss.append(MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:]).cpu().detach().numpy())
            H, _ = Embedder(data)
            H = torch.reshape(H, (data.shape[0], seq_len, hidden_dim))
            H_hat_supervise, _ = Supervisor(H)
            H_hat_supervise = torch.reshape(H_hat_supervise, (data.shape[0], seq_len, hidden_dim))
            trainLoss.append(MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:]).cpu().detach().numpy())
            print('step: '+ str(e) + '/' + str(epoch) + ', s_loss: ' + str(np.sqrt(G_loss_S.cpu().detach().numpy())))

print('Finish Training with Supervised Loss Only')

Start Training with Supervised Loss Only
step: 1/1000, s_loss: 0.23617235
step: 2/1000, s_loss: 0.22093038
step: 3/1000, s_loss: 0.21614064
step: 4/1000, s_loss: 0.2039919
step: 5/1000, s_loss: 0.19177713
step: 6/1000, s_loss: 0.18292084
step: 7/1000, s_loss: 0.171298
step: 8/1000, s_loss: 0.15583871
step: 9/1000, s_loss: 0.15112843
step: 10/1000, s_loss: 0.13194373
step: 11/1000, s_loss: 0.12879778
step: 12/1000, s_loss: 0.11359999
step: 13/1000, s_loss: 0.105906986
step: 14/1000, s_loss: 0.0989981
step: 15/1000, s_loss: 0.087472975
step: 16/1000, s_loss: 0.08264932
step: 17/1000, s_loss: 0.08180722
step: 18/1000, s_loss: 0.071458064
step: 19/1000, s_loss: 0.0706079
step: 20/1000, s_loss: 0.063812055
step: 21/1000, s_loss: 0.061262053
step: 22/1000, s_loss: 0.05697287
step: 23/1000, s_loss: 0.05626616
step: 24/1000, s_loss: 0.050291013
step: 25/1000, s_loss: 0.04892962
step: 26/1000, s_loss: 0.047387708
step: 27/1000, s_loss: 0.045408387
step: 28/1000, s_loss: 0.040520046
step: 29/100

In [98]:
fig = px.line(trainLoss)
fig.add_trace(go.Scatter(y=valLoss, mode="lines"))
fig.show()

In [101]:
print('Start Joint Training')
for e in range(epoch):
    for kk in range(2):
        X = next(iter(loader))
        random_data = random_generator(batch_size=batch_size, z_dim=dim,
                                        T_mb=extract_time(data)[0], max_seq_len=extract_time(data)[1])

        # Generator Training
        ## Train Generator
        z = torch.tensor(np.array(random_data), device=device)
        z = z.float()

        e_hat, _ = Generator(z)
        e_hat = torch.reshape(e_hat, (batch_size, seq_len, hidden_dim))

        H_hat, _ = Supervisor(e_hat)
        H_hat = torch.reshape(H_hat, (batch_size, seq_len, hidden_dim))

        Y_fake = Discriminator(H_hat)
        Y_fake = torch.reshape(Y_fake, (batch_size, seq_len, 1))

        x_hat, _ = Recovery(H_hat)
        x_hat = torch.reshape(x_hat, (batch_size, seq_len, dim))

        H, _ = Embedder(X.float())
        H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

        H_hat_supervise, _ = Supervisor(H)
        H_hat_supervise = torch.reshape(H_hat_supervise, (batch_size, seq_len, hidden_dim))

        Generator.zero_grad()
        Supervisor.zero_grad()
        Discriminator.zero_grad()
        Recovery.zero_grad()

        # line 267 of original implementation:
        # G_loss_U, G_loss_S, G_loss_V
        G_loss_S = MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:])

        G_loss_U = binary_cross_entropy_loss(torch.ones_like(Y_fake), Y_fake)

        G_loss_V1 = torch.mean(torch.abs((torch.std(x_hat, [0], unbiased = False)) + 1e-6 - (torch.std(X, [0]) + 1e-6)))
        G_loss_V2 = torch.mean(torch.abs((torch.mean(x_hat, [0]) - (torch.mean(X, [0])))))
        G_loss_V = G_loss_V1 + G_loss_V2

        # doing a backward step for each loss should result in gradients accumulating
        # so we should be able to optimize them jointly
        G_loss_S.backward(retain_graph=True)#
        G_loss_U.backward(retain_graph=True)
        G_loss_V.backward(retain_graph=True)#


        generator_optimizer.step()
        supervisor_optimizer.step()
        discriminator_optimizer.step()
        # Train Embedder
        ## line 270: we only optimize E_loss_T0
        ## E_loss_T0 = just mse of x and x_tilde
        # but it calls E_solver which optimizes E_loss, which is a sum of
        # E_loss0 and 0.1* G_loss_S
        MSE_loss = nn.MSELoss()

        H, _ = Embedder(X.float())
        H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

        X_tilde, _ = Recovery(H)
        X_tilde = torch.reshape(X_tilde, (batch_size, seq_len, dim))

        E_loss_T0 = MSE_loss(X, X_tilde)
        E_loss0 = 10 * torch.sqrt(MSE_loss(X, X_tilde))

        H_hat_supervise, _ = Supervisor(H)
        H_hat_supervise = torch.reshape(H_hat_supervise, (batch_size, seq_len, hidden_dim))

        G_loss_S = MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:])
        E_loss = E_loss0  + 0.1 * G_loss_S

        G_loss_S.backward(retain_graph=True)
        E_loss_T0.backward()

        Embedder.zero_grad()
        Recovery.zero_grad()
        Supervisor.zero_grad()

        embedder_optimizer.step()
        recovery_optimizer.step()
        supervisor_optimizer.step()
    # train Discriminator
    for batch_index, X in enumerate(loader):
        random_data = random_generator(batch_size=batch_size, z_dim=dim,
                                        T_mb=extract_time(data)[0], max_seq_len=extract_time(data)[1])

        z = torch.tensor(np.array(random_data), device=device)
        z = z.float()

        H, _ = Embedder(X)
        H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

        Y_real = Discriminator(H)
        Y_real = torch.reshape(Y_real, (batch_size, seq_len, 1))

        e_hat, _ = Generator(z)
        e_hat = torch.reshape(e_hat, (batch_size, seq_len, hidden_dim))

        Y_fake_e = Discriminator(e_hat)
        Y_fake_e = torch.reshape(Y_fake_e, (batch_size, seq_len, 1))

        H_hat, _ = Supervisor(e_hat)
        H_hat = torch.reshape(H_hat, (batch_size, seq_len, hidden_dim))

        Y_fake = Discriminator(H_hat)
        Y_fake = torch.reshape(Y_fake, (batch_size, seq_len, 1))

        x_hat, _ = Recovery(H_hat)
        x_hat = torch.reshape(x_hat, (batch_size, seq_len, dim))

        Generator.zero_grad()
        Supervisor.zero_grad()
        Discriminator.zero_grad()
        Recovery.zero_grad()
        Embedder.zero_grad()

        # logits first, then targets
        # D_loss_real(Y_real, torch.ones_like(Y_real))
        D_loss_real = nn.BCEWithLogitsLoss()
        DLR = D_loss_real(Y_real, torch.ones_like(Y_real))

        D_loss_fake = nn.BCEWithLogitsLoss()
        DLF = D_loss_fake(Y_fake, torch.zeros_like(Y_fake))

        D_loss_fake_e = nn.BCEWithLogitsLoss()
        DLF_e = D_loss_fake_e(Y_fake_e, torch.zeros_like(Y_fake_e))

        D_loss = DLR + DLF + gamma * DLF_e

        # D_loss.backward(retain_graph=True)

        # discriminator_optimizer.step()

        # check discriminator loss before updating
        check_d_loss = D_loss
        if (check_d_loss > 0.15):
            D_loss.backward(retain_graph=True)
            discriminator_optimizer.step()

        print('step: '+ str(e) + '/' + str(epoch) +
                ', D_loss: ' + str(D_loss.cpu().detach().numpy()) +
                ', G_loss_U: ' + str(G_loss_U.cpu().detach().numpy()) +
                ', G_loss_S: ' + str(G_loss_S.cpu().detach().numpy()) +
                ', E_loss_t0: ' + str(np.sqrt(E_loss0.cpu().detach().numpy()))
                )
print('Finish Joint Training')

A saída de streaming foi truncada nas últimas 5000 linhas.
step: 487/1000, D_loss: 0.9377502, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.95831805, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.9808192, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.9808941, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.9597231, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.93376005, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.92511535, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.9213763, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_loss: 0.9402349, G_loss_U: 1.2453465, G_loss_S: 1.6412022e-07, E_loss_t0: 2.1953337
step: 487/1000, D_lo

KeyboardInterrupt: 

In [104]:
torch.save(Embedder.state_dict(), 'Embedder.pth')
torch.save(Supervisor.state_dict(), 'Supervisor.pth')
torch.save(Recovery.state_dict(), 'Recovery.pth')
torch.save(Generator.state_dict(), 'Generator.pth')
torch.save(Discriminator.state_dict(), 'Discriminator.pth')

In [20]:
Embedder = Time_GAN_module(input_size=z_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=num_layers)
Recovery = Time_GAN_module(input_size=hidden_dim, output_size=dim, hidden_dim=hidden_dim, n_layers=n_layers)
Generator = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
Supervisor = Time_GAN_module(input_size=hidden_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers-1)
Discriminator = Time_GAN_module(input_size=hidden_dim, output_size=1,
                                hidden_dim=hidden_dim, n_layers=n_layers, activation=nn.Identity)

Embedder.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Embedder.pth',
                                    map_location=torch.device(device)))
Supervisor.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Supervisor.pth',
                                    map_location=torch.device(device)))
Generator.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Generator.pth',
                                    map_location=torch.device(device)))
Discriminator.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Discriminator.pth',
                                    map_location=torch.device(device)))
Recovery.load_state_dict(torch.load('/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/timeGAN-for-HAR/Recovery.pth',
                                    map_location=torch.device(device)))

<All keys matched successfully>

# Discriminative score

In [21]:
class Discri(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Discri, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True, device=device)
        self.fc = nn.Linear(hidden_dim, 1, device=device)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, seq_lengths):
        packed_input = nn.utils.rnn.pack_padded_sequence(x, seq_lengths, batch_first=True, enforce_sorted=False)
        _, h_n = self.gru(packed_input)
        logits = self.fc(h_n[-1])
        y_hat = self.sigmoid(logits)
        return logits, y_hat

In [22]:
hidden_dim = 3
data_dim = 6
disc = Discri(input_dim=data_dim, hidden_dim=hidden_dim)

In [45]:
random_data = random_generator(batch_size=3661, z_dim=dim,
                                T_mb=extract_time(data)[0], max_seq_len=extract_time(data)[1])
c = torch.tensor(np.array(random_data))
z = z.float()

e_hat, _ = Generator(z)
e_hat = torch.reshape(e_hat, (3661, seq_len, 24))

H_hat, _ = Supervisor(e_hat)
H_hat = torch.reshape(H_hat, (3661, seq_len, 24))

x_hat, _ = Recovery(H_hat)
x_hat = torch.reshape(x_hat, (3661, seq_len, dim)).detach()

In [46]:
from torch.utils.data import DataLoader, TensorDataset, random_split
def train_test_divide(dataX, dataX_hat, dataT, test_split=0.2):
    # Combine real and fake data
    data = torch.cat((dataX, dataX_hat), axis=0)
    labels = torch.cat((torch.ones(len(dataX)), torch.zeros(len(dataX_hat))), axis=0)
    lengths = torch.cat((dataT, dataT), axis=0)  # Use same lengths for real and fake

    # Create dataset
    dataset = TensorDataset(data, labels, lengths)

    # Split into train/test
    test_size = int(len(dataset) * test_split)
    train_size = len(dataset) - test_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    return train_dataset, test_dataset

In [47]:
# Parameters
hidden_dim = max(int(data[0].shape[1] / 2), 1)
iterations = 100
batch_size = 128
learning_rate = 0.001

# Prepare data
No = len(data)
data_dim = data[0].shape[1]

# Compute sequence lengths
dataT = [len(seq) for seq in data]

# Get the maximum sequence length
Max_Seq_Len = max(dataT)

# Divide into training and test sets
train_dataset, test_dataset = train_test_divide(data, x_hat, torch.tensor(dataT))

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [48]:
disc = Discri(input_dim=data_dim, hidden_dim=hidden_dim)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(disc.parameters(), lr=0.001)

# Training
disc.train()
for epoch in tqdm(range(iterations)):
    for real_data, labels, seq_lengths in train_loader:
        # Forward pass
        logits, predictions = disc(real_data, seq_lengths)
        loss = criterion(logits.squeeze(), labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|██████████| 100/100 [00:57<00:00,  1.74it/s]


In [49]:
from sklearn.metrics import accuracy_score

In [50]:
# Testing
disc.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for real_data, labels, seq_lengths in test_loader:
        logits, predictions = disc(real_data, seq_lengths)
        y_pred.extend(predictions.squeeze().cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Calculate accuracy
y_pred = np.array(y_pred)
y_true = np.array(y_true)
accuracy = accuracy_score(y_true, y_pred > 0.5)

# Discriminative score
disc_score = np.abs(0.5 - accuracy)

In [51]:
disc_score

0.5

In [57]:
px.line(data[1110])